In [1]:
from src.Base.Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from src.Implicit.FeatureCombinedImplicitALSRecommender import FeatureCombinedImplicitALSRecommender
from src.Utils.confidence_scaling import linear_scaling_confidence
from src.Utils.load_ICM import load_ICM
from src.Utils.load_URM import load_URM
import numpy as np

URM_all = load_URM("../../in/data_train.csv")
ICM_all = load_ICM("../../in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample

URMs_train = []
URMs_validation = []


for k in range(5):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)

evaluator_validation = K_Fold_Evaluator_MAP(URMs_validation, cutoff_list=[10], verbose=False)


In [ ]:
profile_length = np.ediff1d(URM_train.indptr)
block_size = int(len(profile_length)*0.1)
sorted_users = np.argsort(profile_length)


In [ ]:
for group_id in range(0, 10):

    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]

    print("Group {}, average p.len {:.2f}, min {}, max {}".format(group_id,
        users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))


In [ ]:
recommenders = []

for index in range(len(URMs_train)):
    recommenders.append(
        FeatureCombinedImplicitALSRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICM_all,
            verbose=True
        )
    )
    recommenders[index].fit(
        factors=int(factors),
        regularization=0.01,
        use_gpu=False,
        iterations=int(epochs),
        num_threads=4,
        confidence_scaling=linear_scaling_confidence,
        **{
            'URM' : {"alpha":urm_alpha},
            'ICM': {"alpha": icm_alpha}
        }
    )

    result = evaluator_validation.evaluateRecommender(recommenders)
    results.append(result)
    return sum(result) / len(result)

In [ ]:
MAP_hybrid_per_group = []
MAP_super_hyberid_per_group = []
MAP_p3alpha_per_group = []
MAP_featurecombinationRP3beta_per_group = []
MAP_pureSVDitem_per_group = []
MAP_easer_per_group = []
MAP_topPop_per_group = []
cutoff = 10

for group_id in range(0, 10):

    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]

    print("Group {}, average p.len {:.2f}, min {}, max {}".format(group_id,
        users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))


    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert = True)
    users_not_in_group = sorted_users[users_not_in_group_flag]

    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users = users_not_in_group)


    results, _ = evaluator_test.evaluateRecommender(p3alpha_featurecombinedrp3beta_recommender)
    MAP_hybrid_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(p3alpha_recommender)
    MAP_p3alpha_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(rp3betaCBF_recommender)
    MAP_featurecombinationRP3beta_per_group.append(results[cutoff]["MAP"])

    #results, _ = evaluator_test.evaluateRecommender(pureSVDitem_recommender)
    #MAP_pureSVDitem_per_group.append(results[cutoff]["MAP"])

    # results, _ = evaluator_test.evaluateRecommender(easer_recommender)
    # MAP_easer_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(p3alpha_featurecombinedrp3beta_rp3betacf_recommender)
    MAP_super_hyberid_per_group.append(results[cutoff]["MAP"])

    results, _ = evaluator_test.evaluateRecommender(topPop)
    MAP_topPop_per_group.append(results[cutoff]["MAP"])



In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline

pyplot.plot(MAP_hybrid_per_group, label=p3alpha_featurecombinedrp3beta_recommender.RECOMMENDER_NAME)
pyplot.plot(MAP_p3alpha_per_group, label=p3alpha_recommender.RECOMMENDER_NAME)
pyplot.plot(MAP_featurecombinationRP3beta_per_group, label=RP3betaCBFRecommender.RECOMMENDER_NAME)
pyplot.plot(MAP_super_hyberid_per_group, label=p3alpha_featurecombinedrp3beta_rp3betacf_recommender.RECOMMENDER_NAME)
#pyplot.plot(MAP_pureSVDitem_per_group, label=pureSVDitem_recommender.RECOMMENDER_NAME)
# pyplot.plot(MAP_easer_per_group, label=easer_recommender.RECOMMENDER_NAME)
pyplot.plot(MAP_topPop_per_group, label="topPop")
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()



In [ ]:
URM_train


In [ ]:
from src.Base.Recommender_utils import ratingMatrixTopK
from src.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from src.Utils.ICM_preprocessing import *
ICM_combined = combine(ICM=ICM_all, URM = URM_train)
rp3betaCBF_recommender = RP3betaCBFRecommender(URM_train=URM_train, ICM_train=ICM_combined)
rp3betaCBF_recommender.fit(topK=586, alpha=0.4228, beta=0.1918, implicit=False)

URM_new = ratingMatrixTopK(rp3betaCBF_recommender.URM_train.dot(rp3betaCBF_recommender.W_sparse), k=10)

URM_new

In [ ]:
URM_in = rp3betaCBF_recommender.URM_train.dot(rp3betaCBF_recommender.W_sparse)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

from bayes_opt import BayesianOptimization

In [ ]:
tuning_params = {
    "matrixTopK": (1, 100),
    "alpha": (0.1, 0.9),
    "beta": (0.1, 0.9),
    "cfTopK": (10, 600)
}


def BO_func(
        matrixTopK,
        alpha,
        beta,
        cfTopK
):

    URM_new = ratingMatrixTopK(URM_in, k=int(matrixTopK))
    rp3betaCBF_recommender = RP3betaCBFRecommender(URM_train=URM_train, ICM_train=URM_new.T, verbose=False)
    rp3betaCBF_recommender.fit(alpha=alpha, beta=beta,topK=int(cfTopK), implicit=False)
    result_dict, _ = evaluator_test.evaluateRecommender(rp3betaCBF_recommender)

    return result_dict[10]["MAP"]


optimizer = BayesianOptimization(
    f=BO_func,
    pbounds=tuning_params,
    verbose=5,
    random_state=5,
)

optimizer.maximize(
    init_points=10,
    n_iter=8,
)